In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-dataset.zip

--2022-08-04 23:40:32--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-dataset.zip [following]
--2022-08-04 23:40:32--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc51ffd96cef39a5c57173e6ab71.dl.dropboxusercontent.com/cd/0/inline/BqYXblJvFelwzjcqRMYXCM9W5790e_YSZuD7WQsnlA64vumTi402ZPvVUjmxbZMaXFHElBN5OcDbT_A66v6faP8nH_l0jsiPP2HLpNLy4neIMTbuB46jgXoaC7dUeb3tkrGHJ1HPUk6hdEKlo3wM90nA4FfYq0FKG_9ufe1om1vxlg/file# [following]
--2022-08-04 23:40:33--  https://uc51ffd96cef39a5c57173e6ab71.dl.dropboxusercontent.com/cd/0/inline/BqYXblJvFelwzjcqRMYXCM9W5790e_YSZuD7WQsnlA64vumTi402Z

In [2]:
!unzip -q 'movie-dataset.zip'

In [3]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, SimpleRNNCell, Embedding, Dropout

In [4]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
      lambda text, label: (regex_replace(text, '<br />',' '), label),)

In [5]:
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b"Meryl Streep is such a genius. Well, at least as an actress. I know she's been made fun of for doing a lot of roles with accents, but she nails the accent every time. Her performance as Lindy Chamberlain was inspiring. Mrs. Chamberlain, as portrayed here, was not particularly likable, nor all that smart. But that just makes Streep's work all the more remarkable. I think she is worth all 10 or so of her Oscar nominations. About the film, well, there were a couple of interesting things. I don't know much about Australia, but the theme of religious bigotry among the general public played a big part in the story. I had largely missed this when I first saw the film some years ago, but it came through loud and clear yesterday. And it seems the Australian press is just as accomplished at misery-inducing pursuit and overkill as their American colleagues. A pretty good film. A bit different. Grade: B"
1


In [6]:
model = Sequential()
model.add(Input(shape=(1,),dtype='string'))

In [7]:
max_tokens = 1000
max_len = 100

vectorized_layer = TextVectorization(max_tokens = max_tokens,
                                     output_mode = 'int',
                                     output_sequence_length = max_len,)





In [8]:
train_text = train_data.map(lambda text, label : text)
vectorized_layer.adapt(train_text)

model.add(vectorized_layer)

In [9]:
# creating an embedding layer

model.add(Embedding(max_tokens + 1, 128))
# rnn = RNN(SimpleRNNCell(64), return_sequences = False, return_state = False) # can change it from here
model.add(LSTM(64))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [10]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [11]:
model.fit(train_data, epochs = 10)

Epoch 1/10
782/782 [==============================] - 78s 95ms/step - loss: 0.5428 - accuracy: 0.7173
Epoch 2/10
782/782 [==============================] - 74s 94ms/step - loss: 0.4367 - accuracy: 0.8000
Epoch 3/10
782/782 [==============================] - 73s 93ms/step - loss: 0.4026 - accuracy: 0.8172
Epoch 4/10
782/782 [==============================] - 74s 95ms/step - loss: 0.3763 - accuracy: 0.8345
Epoch 5/10
782/782 [==============================] - 72s 92ms/step - loss: 0.3592 - accuracy: 0.8422
Epoch 6/10
782/782 [==============================] - 75s 96ms/step - loss: 0.3399 - accuracy: 0.8544
Epoch 7/10
782/782 [==============================] - 73s 93ms/step - loss: 0.3255 - accuracy: 0.8620
Epoch 8/10
782/782 [==============================] - 75s 96ms/step - loss: 0.3130 - accuracy: 0.8678
Epoch 9/10
782/782 [==============================] - 75s 95ms/step - loss: 0.3084 - accuracy: 0.8691
Epoch 10/10
782/782 [==============================] - 74s 94ms/step - loss: 0.291

In [12]:
model.evaluate(test_data)

782/782 [==============================] - 26s 32ms/step - loss: 0.5664 - accuracy: 0.7798


[0.5663663148880005, 0.7797999978065491]

In [13]:
model.predict(['I loved the movie'])

array([[0.95213044]], dtype=float32)